# Stock Sentiment Analysis Project

## Gathering the Article Data for our Stocks
Utilizing the News API service, we gather articles referencing the Stock Symbols that we are interested in. We take the JSON data returned, clean it, and store into our sqlite database for future reference.

In [ ]:
import sys
from news_finders import content_scraper
from news_finders import newsApi
from utils import file_utils
from utils import sqlite_utils
from datetime import datetime, timedelta
  

STOCKS = ["AAPL", "ABBV", "TXN"]


date= datetime.today() - timedelta(days=28)

reformatedDate= str(date.month) + "-" + str(date.day) + "-" + str(date.year)


articles = []
articles = newsApi.getArticlesForMultipleStocks(STOCKS, reformatedDate)



conn= sqlite_utils._createConnection(r"/var/stockSA/stockSentiment.db")

if conn is None: 
    print("Failed to open database connection")
    sys.exit(1)

sqlite_utils.insertStockArticles(conn, articles)
print(sqlite_utils._findStockArticlesForSymbol(conn, "AAPL")[0])

## Gathering the Historical Price Data for our Stocks
Utilizing the Yahoo Financials API, we gather stock finance data per each company that we are interested in and store that data in a Stock Pricing Database. 

In [ ]:
import sys
from yahoofinancials import YahooFinancials
from utils import file_utils
from utils import sqlite_utils
from datetime import datetime, timedelta


# connecting to the database
conn = sqlite_utils._createConnection(r"/var/stockSA/stockPricing.db")
if conn is None:
    print("Failed to open database connection")
    sys.exit(1)
sqlite_utils._createStockPricingTable(conn)

while True :
    ticker = input('Enter a stock symbol or to quit program, type quit: ')
    if ticker == 'quit' :
        quit()

    try: 
        yahoo_financials = YahooFinancials(ticker) 
        # call to the YahooFinance tool function
        historicalStockPrices = yahoo_financials.get_historical_price_data(
                "2020-06-09", "2020-07-08", "daily")
        
        # creating our variable prices, storing all of the data in a list
        if historicalStockPrices[ticker] is None:
            print('Failed to get stock prices')
            continue
        
        prices = historicalStockPrices[ticker]['prices']
        sqlite_utils.insertPrices(conn, ticker, prices)	

    except Exception as e: 
    	print("Invalid StockSymbol, no data ", e)
    	continue
    
    print(prices)

Enter a stock symbol or to quit program, type quit: AAPL
Failed to create price data UNIQUE constraint failed: stockPricing.stockSymbol, stockPricing.date {'date': 1591709400, 'high': 86.40249633789062, 'low': 83.00250244140625, 'open': 83.03500366210938, 'close': 85.99749755859375, 'volume': 147712400, 'adjclose': 85.69487762451172, 'formatted_date': '2020-06-09'}
Failed to create price data UNIQUE constraint failed: stockPricing.stockSymbol, stockPricing.date {'date': 1591795800, 'high': 88.69249725341797, 'low': 86.52249908447266, 'open': 86.9749984741211, 'close': 88.20999908447266, 'volume': 166651600, 'adjclose': 87.89958953857422, 'formatted_date': '2020-06-10'}
Failed to create price data UNIQUE constraint failed: stockPricing.stockSymbol, stockPricing.date {'date': 1591882200, 'high': 87.76499938964844, 'low': 83.87000274658203, 'open': 87.32749938964844, 'close': 83.9749984741211, 'volume': 201662400, 'adjclose': 83.67949676513672, 'formatted_date': '2020-06-11'}
Failed to cr

Enter a stock symbol or to quit program, type quit: quit
Invalid StockSymbol, no data  'quit'
